In [2]:
# import everything ml
import torch
from torch.utils.data import DataLoader, TensorDataset
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim
import torch.nn.functional as F
import pandas as pd
import re
import string
from wordcloud import WordCloud
import nltk
from collections import Counter
import contractions
from tqdm import tqdm
import math
from torch.utils.data import Dataset, DataLoader
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split


In [3]:
true_dataset = pd.read_csv("./data/fakenews/True.csv")
false_dataset = pd.read_csv("./data/fakenews/Fake.csv")

In [4]:
def combine_and_randomize_datasets(true_dataset, false_dataset):
    true_arr = np.ones(len(true_dataset))
    false_arr = np.zeros(len(false_dataset))

    true_dataset['real'] = true_arr
    false_dataset['real'] = false_arr
    combined_dataset = pd.concat([true_dataset, false_dataset], ignore_index=True)
    combined_dataset = combined_dataset.sample(frac=1).reset_index(drop=True)

    return combined_dataset

combined = combine_and_randomize_datasets(true_dataset, false_dataset)

In [5]:


def clean_text(text):
    text = contractions.fix(text)                         # Expand contractions
    text = text.encode("ascii", errors="ignore").decode() # Remove non-ASCII chars
    text = text.lower()                                   # Lowercase
    text = re.sub(r"http\S+|www.\S+", "", text)           # Remove URLs
    text = re.sub(r"<.*?>", "", text)                     # Remove HTML tags
    text = re.sub(r"[^a-z\s]", "", text)                  # Keep only letters and spaces
    text = re.sub(r"\b\w{1}\b", "", text)                 # Remove single-letter words
    text = re.sub(r"\s+", " ", text).strip()              # Remove extra spaces
    words = text.split()
    # words = [word for word in words if word not in stop_words]  # Remove stopwords
    return " ".join(words)



In [6]:
words_dict = {}

# def process_words_into_dict(text, words_dict):
#     for word in text.split():
#         if word not in words_dict:
#             words_dict[word] = 1
#         else:
#             words_dict[word] += 1

# for text in combined['text']:
#     text = clean_text(text)
# tfidf = calculate_tfidf(combined['text'])

X = combined['text']
y = combined['real'].to_list()

test_x = X[:20]
X = X[20:]

test_y=y[:20]
y=y[20:]

vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1,2), stop_words='english')
X_tfidf = vectorizer.fit_transform(X)  # shape (n_samples, n_features)

X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

# --- Convert to dense and then to PyTorch tensors
X_train_tensor = torch.tensor(X_train.toarray(), dtype=torch.float32)
X_test_tensor = torch.tensor(X_test.toarray(), dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)


In [7]:
# --- PyTorch Dataset
class NewsDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y.unsqueeze(1)  # Per sample

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_dataset = NewsDataset(X_train_tensor, y_train_tensor)
test_dataset = NewsDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)


In [8]:
# --- Simple Feedforward Model
# class FakeNewsClassifier(nn.Module):
#     def __init__(self, input_dim):
#         super().__init__()
#         self.fc1 = nn.Linear(input_dim, 100)
#         self.fc2 = nn.Linear(100, 1)

#     def forward(self, x):
#         x = F.relu(self.fc1(x))
#         x = torch.sigmoid(self.fc2(x))
#         return x

class FakeNewsClassifier(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.classifier = nn.Sequential(
            nn.Linear(input_dim, 100),
            nn.ReLU(),
            nn.Linear(100, 1000),
            nn.ReLU(),
            nn.Linear(1000,1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.classifier(x)
device = torch.device("cuda" if torch.cuda.is_available else "cpu")
model = FakeNewsClassifier(input_dim=5000).to(device)
loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [9]:
# --- Training Loop
for epoch in range(50):
    model.train()
    total_loss = 0
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        outputs = model
        outputs = model(batch_X)
        loss = loss_fn(outputs, batch_y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1} - Loss: {total_loss:.4f}")


Epoch 1 - Loss: 72.5067
Epoch 2 - Loss: 12.6047
Epoch 3 - Loss: 7.5630
Epoch 4 - Loss: 4.9280


Epoch 5 - Loss: 2.7961
Epoch 6 - Loss: 2.2009
Epoch 7 - Loss: 2.1395
Epoch 8 - Loss: 1.7853
Epoch 9 - Loss: 0.9046
Epoch 10 - Loss: 1.4824
Epoch 11 - Loss: 1.0486
Epoch 12 - Loss: 0.5332
Epoch 13 - Loss: 0.5083
Epoch 14 - Loss: 0.5019
Epoch 15 - Loss: 3.9367
Epoch 16 - Loss: 4.7794
Epoch 17 - Loss: 3.9189
Epoch 18 - Loss: 4.0341
Epoch 19 - Loss: 1.0742
Epoch 20 - Loss: 0.6277
Epoch 21 - Loss: 1.7112
Epoch 22 - Loss: 0.5897
Epoch 23 - Loss: 1.0150
Epoch 24 - Loss: 0.3680
Epoch 25 - Loss: 0.3715
Epoch 26 - Loss: 0.4736
Epoch 27 - Loss: 0.4903
Epoch 28 - Loss: 0.3508
Epoch 29 - Loss: 0.4230
Epoch 30 - Loss: 0.5525
Epoch 31 - Loss: 0.4194
Epoch 32 - Loss: 0.6364
Epoch 33 - Loss: 0.4967
Epoch 34 - Loss: 0.3984
Epoch 35 - Loss: 0.4956
Epoch 36 - Loss: 0.4587
Epoch 37 - Loss: 0.4894
Epoch 38 - Loss: 0.4548
Epoch 39 - Loss: 0.7010
Epoch 40 - Loss: 0.5235
Epoch 41 - Loss: 0.5429
Epoch 42 - Loss: 0.4988
Epoch 43 - Loss: 1.0397
Epoch 44 - Loss: 0.9214
Epoch 45 - Loss: 1.0818
Epoch 46 - Loss: 5.90

In [10]:
# --- Evaluation
model.eval()
with torch.no_grad():
    X_test_tensor = X_test_tensor.to(device)  # Ensure inputs are on model's device
    y_test_tensor = y_test_tensor.to(device)  # Ensure labels are on the same device

    preds = model(X_test_tensor).squeeze()
    #print(f"average pred: {preds.mean().item()}")
    preds_label = (preds > 0.5).int()
    accuracy = (preds_label == y_test_tensor.int()).float().mean()
    print(f"Test Accuracy: {accuracy:.4f}")

Test Accuracy: 0.9929


In [11]:
def predict_news_article(article_text, vectorizer, model):
    model.eval()
    with torch.no_grad():
        # Vectorize input string
        tfidf_vector = vectorizer.transform([article_text])
        tfidf_tensor = torch.tensor(tfidf_vector.toarray(), dtype=torch.float32).to(device)

        # Forward pass
        output = model(tfidf_tensor)
    
        return output, output > 0.5

# test_article = "The government has announced a new plan to build solar-powered roads by 2050..."
# test_article=false_dataset['text'][123]
idx = 9
test_article = test_x[idx]
test_article_pred = test_y[idx]
print(f"pred {test_article_pred} testing article: {test_article}")

prediction, is_true = predict_news_article(test_article, vectorizer, model)
print(f"Prediction: {prediction} \n Is True {is_true} was actually: {test_article_pred>0.5}")

pred 1.0 testing article: NEW YORK (Reuters) - A new rule aimed at protecting retirement funds will result in up to $31.5 billion in compliance costs for the brokerage industry over the next decade but produce even bigger gains for retirees, the U.S. Labor Department said on Wednesday. In its final fiduciary standard rule released on Wednesday, the Department predicted industry compliance costs of between $10 billion and $31.5 billion over the next 10 years. Individual retirement account investors will gain between $33 billion and $36 billion over the next decade as a result of the rule, and between $66 billion and $76 billion over the next 20 years, the Department said. 
Prediction: tensor([[1.]], device='cuda:0') 
 Is True tensor([[True]], device='cuda:0') was actually: True
